## 🍓 basic resnet50

In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
test_dir = '/opt/ml/input/data/eval'

In [3]:
import torch
import torch.utils.data as data
import pandas as pd
import numpy as np
from glob import glob

def file_load(opt):
    
    #data_path = []
    #f = open("{0}.txt".format(opt), 'r')
    files = glob.glob("/opt/ml/input/data/train/images/*/*") 
    #while True:
        #line = f.readline()
        #if not line: break
        #data_path.append(line[:-1])
    #f.close()
    return files

class CustomDataset(data.Dataset):
    def __init__(self, opt_data):
        super(CustomDataset, self).__init__()

        """
        opt_data : 'train', 'validation'
        
        """
        #self.file_list = file_load('/opt/ml/input/data/train/train_path.csv')
        y = pd.read_csv('/opt/ml/input/data/train/train_path.csv', index_col=0)
        self.y = y.values
        
    def __getitem__(self, index):
        
        x = np.load(self.file_list[index])
        self.x_data = torch.from_numpy(x).float()
        self.y_data = torch.from_numpy(self.y[index]).float()
        return self.x_data, self.y_data

    def __len__(self):
        return len(self.y)
        
if __name__ == "__main__":
    a = CustomDataset('train')

## call resnet50

In [4]:
from tensorflow.keras.applications import ResNet50

model = ResNet50(include_top=True, weights=None, input_shape=(64, 48, 3), pooling=max, classes=18)

2021-08-26 12:17:22.363055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 12:17:22.364413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-26 12:17:22.364488: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2021-08-26 12:17:22.364500: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if 

In [5]:
from keras.applications.imagenet_utils import decode_predictions 

In [6]:
def setup_gpus():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.experimental.set_visible_devices(gpus[0],'GPU')
            tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1500)])
        except RuntimeError as e:
            print(e)

In [7]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print('compile end')


compile end


## call image

In [8]:
from PIL import Image
from numpy import genfromtxt
import gzip
import _pickle
from glob import glob
import numpy as np
import pandas as pd
import imageio
import cv2


def dir_to_dataset(glob_files, loc_train_labels=""):
    print("Gonna process:\n\t %s"%glob_files)
    dataset = []
    for file_count, file_name in enumerate( sorted(glob(glob_files)) ):
        img = imageio.imread(file_name, pilmode='RGB')
        img = cv2.resize(img, (48, 64))
        #pixels = [f[0] for f in list(img.getdata())]
        dataset.append(img)
        if file_count % 1000 == 0:
            print("\t %s files processed"%file_count)
    print('done')
    print(img.shape)
        
    # outfile = glob_files+"out"
    # np.save(outfile, dataset)
    if len(loc_train_labels) > 0:
        df = pd.read_csv(loc_train_labels, names = ["class"])
        return np.array(dataset), np.array(df["class"])
    else:
        return np.array(dataset)
    
Data1, y1 = dir_to_dataset("/opt/ml/input/data/train/images/*/*","/opt/ml/input/data/train/train_path.csv")
Data2, y2 = dir_to_dataset("/opt/ml/input/data/eval/images/*","/opt/ml/input/data/eval/info.csv")

print('yup')

# Data and labels are read 
train_num = 17000
valid_num = 1000
test_num = 900
submit_num  = 12600

train_set_x = Data1[:train_num]
train_set_y = y1[1:train_num+1]
val_set_x = Data1[17000:17000+valid_num]
val_set_y = y1[17001:17001+valid_num]
test_set_x = Data1[18000:18000+test_num]
test_set_y = y1[18001:18001+test_num]

submit_set_x = Data2[:submit_num]
submit_set_y = y1[1:submit_num+1]

train_set = train_set_x, train_set_y
val_set = val_set_x, val_set_y
test_set = test_set_x, test_set_y

submit_set = submit_set_x, submit_set_y

dataset = [train_set, val_set, test_set, submit_set]

Gonna process:
	 /opt/ml/input/data/train/images/*/*
	 0 files processed
	 1000 files processed
	 2000 files processed
	 3000 files processed
	 4000 files processed
	 5000 files processed
	 6000 files processed
	 7000 files processed
	 8000 files processed
	 9000 files processed
	 10000 files processed
	 11000 files processed
	 12000 files processed
	 13000 files processed
	 14000 files processed
	 15000 files processed
	 16000 files processed
	 17000 files processed
	 18000 files processed
done
(64, 48, 3)
Gonna process:
	 /opt/ml/input/data/eval/images/*
	 0 files processed
	 1000 files processed
	 2000 files processed
	 3000 files processed
	 4000 files processed
	 5000 files processed
	 6000 files processed
	 7000 files processed
	 8000 files processed
	 9000 files processed
	 10000 files processed
	 11000 files processed
	 12000 files processed
done
(64, 48, 3)
yup


## data setting

In [15]:
%%time 
import _pickle, gzip, urllib.request, json
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import np_utils

train_set, valid_set, test_set, submit_set = dataset
    
(train_images, train_labels), (valid_images, valid_labels), (test_images, test_labels), (submit_images, submit_labels) = train_set, valid_set, test_set, submit_set

train_images = train_images.reshape(train_images.shape[0], 64, 48, 3)
valid_images = valid_images.reshape(valid_images.shape[0], 64, 48, 3)
test_images = test_images.reshape(test_images.shape[0], 64, 48, 3)
submit_images = submit_images.reshape(submit_images.shape[0], 64, 48, 3)

train_images = train_images.astype('float32')
valid_images = valid_images.astype('float32')
test_images = test_images.astype('float32')
submit_images = submit_images.astype('float32')

train_images /= 255
valid_images /= 255
test_images /= 255
submit_images /= 255

n_classes = 18
print("Shape before one-hot encoding: ", train_labels.shape)
train_labels = np_utils.to_categorical(train_labels, n_classes)
valid_labels = np_utils.to_categorical(valid_labels, n_classes)
test_labels = np_utils.to_categorical(test_labels, n_classes)
print("Shape after one-hot encoding: ", train_labels.shape)

'''
for i in range(0,10000,2000):
    img = train_set[0][i]
    label = train_set[1][i]
    img_reshape = img.reshape((64,48,3))
    imgplot = plt.imshow(img_reshape)
    print('This is a {}'.format(label))
    plt.show()
'''

Shape before one-hot encoding:  (17000,)
Shape after one-hot encoding:  (17000, 18)
CPU times: user 240 ms, sys: 300 ms, total: 540 ms
Wall time: 542 ms


"\nfor i in range(0,10000,2000):\n    img = train_set[0][i]\n    label = train_set[1][i]\n    img_reshape = img.reshape((64,48,3))\n    imgplot = plt.imshow(img_reshape)\n    print('This is a {}'.format(label))\n    plt.show()\n"

## Training

In [17]:
#es = callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
#save = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
#callback = [es, save]

#hist = model.fit(train_images, train_labels, batch_size=64, epochs=10, validation_data=(valid_images, valid_labels), callbacks=callback)

model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_data=(valid_images, valid_labels))
print('fit end')
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('acc : ', test_acc)

Epoch 1/10
133/133 [==============================] - 373s 3s/step - loss: 1.6886 - accuracy: 0.4982 - val_loss: 6.8036 - val_accuracy: 0.0000e+00
Epoch 2/10
133/133 [==============================] - 372s 3s/step - loss: 0.8879 - accuracy: 0.7095 - val_loss: 3.0328 - val_accuracy: 0.4970
Epoch 3/10
133/133 [==============================] - 371s 3s/step - loss: 0.5980 - accuracy: 0.8046 - val_loss: 1.5788 - val_accuracy: 0.6920
Epoch 4/10
133/133 [==============================] - 372s 3s/step - loss: 0.4400 - accuracy: 0.8529 - val_loss: 1.8269 - val_accuracy: 0.6080
Epoch 5/10
133/133 [==============================] - 371s 3s/step - loss: 0.3183 - accuracy: 0.8938 - val_loss: 2.8904 - val_accuracy: 0.6390
Epoch 6/10
133/133 [==============================] - 370s 3s/step - loss: 0.2775 - accuracy: 0.9071 - val_loss: 2.8878 - val_accuracy: 0.5040
Epoch 7/10
133/133 [==============================] - 371s 3s/step - loss: 0.2301 - accuracy: 0.9216 - val_loss: 1.8387 - val_accuracy: 0.

In [15]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [18]:
pred = model.predict(submit_images)
print(np.argmax(pred[0]))

3


## make submit_file

In [20]:
from tqdm import notebook
import glob
import os
import csv
import re
img_files = sorted(glob.glob("/opt/ml/input/data/eval/images/*"))
search = '/opt/ml/input/data/eval/images/'

for i, word in enumerate(img_files):
    if search in word: 
        img_files[i] = word.strip(search)

#print(img_files)

In [25]:
submit_dic = {}

for i in range(len(pred)):
    submit_dic[img_files[i]] = np.argmax(pred[i])

#print(submit_dic[img_files[0]])

3


In [26]:
df = pd.DataFrame(list(submit_dic.items()),columns=['ImageID', 'ans'])
df.to_csv('/opt/ml/input/data/eval/submit0826.csv', mode='w')